In [8]:
!git clone https://github.com/caseyhird/casey-lm.git

Cloning into 'casey-lm'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 17 (delta 2), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 6.18 KiB | 6.18 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [19]:
!cd /content/casey-lm && pip install -qr requirements.txt

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.4 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.6/717.6 kB 47.5 MB/s eta 0:00:00
 

In [25]:
# Allow imports from the project directory
import sys
sys.path.insert(0, '/content/casey-lm')


In [20]:
import torch

# Set the default tensor type to CUDA (GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [1]:
from data.glue_dataloader import gen_dataloaders
batch_size = 32
max_length = 128

train_dataloader, val_dataloader, vocab_size = gen_dataloaders(batch_size, max_length)

/Users/crhird/Documents/Projects/casey-lm/casey-lm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/crhird/Documents/Projects/casey-lm/casey-lm/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 872/872 [00:00<00:00, 22292.72 examples/s]


In [28]:
import tqdm
import torch
from model.torch_impl.language_model import TorchLanguageModel, LanguageModelConfig

def do_train():
    model = TorchLanguageModel(LanguageModelConfig(**{
        'vocab_size': vocab_size,
        'context_length': max_length,
        'embedding_dim': 8,
        'num_decoder_layers': 3,
        'num_heads': 2,
        'dim_feedforward': 32,
        'dropout': 0.1,
    }))
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(5):
        model.train()
        train_loss = 0.0
        num_train_examples = 0
        for batch in tqdm.tqdm(train_dataloader):
            inputs = batch[:, :-1]
            labels = batch[:, 1:]
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output.view(-1, vocab_size), labels.flatten())
            train_loss += loss.item()
            num_train_examples += labels.numel()
            loss.backward()
            optimizer.step()

        train_loss /= num_train_examples
        print(f'Epoch {epoch+1}, Training Loss: {train_loss:.4f}')

        model.eval()
        with torch.no_grad():
            total_correct = 0
            num_val_examples = 0
            for batch in val_dataloader:
                inputs = batch[:, :-1]
                labels = batch[:, 1:]
                output = model(inputs)
                _, predicted = torch.max(output, dim=2)
                total_correct += (predicted == labels).sum().item()
                num_val_examples += labels.numel()

            accuracy = total_correct / num_val_examples
            print(f'Epoch {epoch+1}, Validation Accuracy: {accuracy:.4f}')